In [108]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
from IPython.display import display
keypoint_connections=[[0,1],
[0,15],
[0,16],
[15,17],
[16,18],
[1,5],
[5,6],
[6,7],
[1,2],
[2,3],
[3,4],
[1,8],
[8,9],
[9,10],
[10,11],
[11,24],
[11,22],
[22,23],
[8,12],
[12,13],
[13,14],
[14,21],
[14,19],
[19,20]
]

frame_jsons = [json.load(open(f"output/{p}")) for p in sorted(os.listdir("output/"))]
num_frames = len(frame_jsons)
total_runners = 8
valid_frame = 165
valid_json = frame_jsons[valid_frame]
valid_frame_coms = []
valid_frame_person_order = list(range(8))
all_frames_person_order = [valid_frame_person_order]*num_frames
for idx,person in enumerate(valid_json["people"]):
    keypoints = person["pose_keypoints_2d"]
    keypoints = np.array(person["pose_keypoints_2d"]).reshape((25,3))
    keypoints = keypoints[keypoints[:,2]>0]
    valid_frame_coms.append([np.mean(keypoints[:,0]), np.mean(keypoints[:,1])])

# backward pass
previous_coms = np.array(valid_frame_coms)
for frame_idx in range(valid_frame-1,-1,-1):
    frame_json = frame_jsons[frame_idx]
    current_coms = []
    for person in frame_json["people"]:
        keypoints = np.array(person["pose_keypoints_2d"]).reshape((25,3))
        keypoints = keypoints[keypoints[:,2]>0]
        current_coms.append(np.array([np.mean(keypoints[:,0]), np.mean(keypoints[:,1])]))
    current_coms = np.array(current_coms)

    current_frame_person_order = []
    current_frame_person_dist = []
    for idx,previous_com in enumerate(previous_coms):
        delta = current_coms - previous_com
        dists = np.sum(np.abs(delta)**2,axis=-1)**(1./2)
        current_dist = np.min(dists)
        closest_com_idx = np.argmin(dists)
        if closest_com_idx not in current_frame_person_order:
            current_frame_person_order.append(closest_com_idx)
            current_frame_person_dist.append(current_dist)
        else:
            #then two coms close to this point, but can only be one
            previous_dist = current_frame_person_dist[current_frame_person_order.index(closest_com_idx)]
            if current_dist < previous_dist:
                # remove previous
                current_frame_person_order[current_frame_person_order.index(closest_com_idx)] = -1
                current_frame_person_order.append(closest_com_idx)
                current_frame_person_dist.append(current_dist)
            else:
                current_frame_person_order.append(-1)
                current_frame_person_dist.append(current_dist)

    for idx in range(previous_coms.shape[0]):
        if current_frame_person_order[idx]>=0:
            previous_coms[idx,:] = current_coms[current_frame_person_order[idx],:]
    all_frames_person_order[frame_idx] = current_frame_person_order

# forward pass
previous_coms = np.array(valid_frame_coms)
for frame_idx in range(valid_frame+1,num_frames,1):
    frame_json = frame_jsons[frame_idx]
    current_coms = []
    for person in frame_json["people"]:
        keypoints = np.array(person["pose_keypoints_2d"]).reshape((25,3))
        keypoints = keypoints[keypoints[:,2]>0]
        current_coms.append(np.array([np.mean(keypoints[:,0]), np.mean(keypoints[:,1])]))
    current_coms = np.array(current_coms)

    current_frame_person_order = []
    current_frame_person_dist = []
    for idx,previous_com in enumerate(previous_coms):
        delta = current_coms - previous_com
        dists = np.sum(np.abs(delta)**2,axis=-1)**(1./2)
        current_dist = np.min(dists)
        closest_com_idx = np.argmin(dists)
        if closest_com_idx not in current_frame_person_order:
            current_frame_person_order.append(closest_com_idx)
            current_frame_person_dist.append(current_dist)
        else:
            #then two coms close to this point, but can only be one
            previous_dist = current_frame_person_dist[current_frame_person_order.index(closest_com_idx)]
            if current_dist < previous_dist:
                # remove previous
                previous_idx = current_frame_person_order.index(closest_com_idx)
                current_frame_person_order[previous_idx] = -1
                current_frame_person_order.append(closest_com_idx)
                current_frame_person_dist.append(current_dist)
            else:
                current_frame_person_order.append(-1)
                current_frame_person_dist.append(current_dist)

    for idx in range(previous_coms.shape[0]):
        if current_frame_person_order[idx]>=0:
            previous_coms[idx,:] = current_coms[current_frame_person_order[idx],:]
    all_frames_person_order[frame_idx] = current_frame_person_order




@widgets.interact(t=(0, len(frame_jsons)-1))
def display_frame(t):
    color_by_person = True
    current_json = frame_jsons[t]
    people = current_json["people"]
    for person_idx in all_frames_person_order[t]:
        if person_idx == -1:
            plt.gca()._get_lines.get_next_color()
        else:
            person = people[person_idx]
            plt.ylim([1000,200])
            plt.xlim([0, 1600])
            keypoints = np.array(person["pose_keypoints_2d"]).reshape((25,3))
            if color_by_person:
                color = plt.gca()._get_lines.get_next_color()
                for keypoint_connection in keypoint_connections:
                    if not (keypoints[keypoint_connection[0],2] == 0 or keypoints[keypoint_connection[1],2] == 0):
                        plt.plot(keypoints[keypoint_connection,0],keypoints[keypoint_connection,1],color=color)
                valid_keypoints = keypoints[keypoints[:,2]>0]
                plt.plot(np.mean(valid_keypoints[:,0]),np.mean(valid_keypoints[:,1]),'o',color=color)
            else:
                plt.gca().set_prop_cycle(None)
                for keypoint_connection in keypoint_connections:
                    if not (keypoints[keypoint_connection[0],2] == 0 or keypoints[keypoint_connection[1],2] == 0):
                        plt.plot(keypoints[keypoint_connection,0],keypoints[keypoint_connection,1])
                    else:
                        plt.gca()._get_lines.get_next_color()


interactive(children=(IntSlider(value=165, description='t', max=330), Output()), _dom_classes=('widget-interac…

In [105]:
for i,v in enumerate(all_frames_person_order):
    print(i,v)

0 [3, 1, 0, 10, 13, 2, 4, 11]
1 [3, 0, 1, -1, 11, 2, 4, 6]
2 [3, 1, 0, 14, 5, 2, 4, 9]
3 [1, 3, 0, -1, 2, 4, 5, 12]
4 [1, 2, -1, 15, 13, 7, 3, 11]
5 [1, 6, -1, 15, 9, 16, 2, 12]
6 [2, 5, -1, 12, 1, -100, 3, 8]
7 [2, 4, -1, 6, 1, 11, 3, 9]
8 [2, 5, 14, 8, 1, 9, 4, 12]
9 [7, 8, 13, 11, 1, 2, 4, 5]
10 [10, 6, 12, 13, 1, 2, 4, 5]
11 [4, 5, 12, 14, 1, 10, 3, 6]
12 [7, 1, 12, 8, 2, 11, 5, 6]
13 [8, 1, 10, 12, 2, 6, 5, 9]
14 [4, 1, 8, 14, 6, 11, 3, 12]
15 [2, 1, 14, 12, 7, 8, 3, 10]
16 [1, 2, 12, 10, 5, 6, 3, 14]
17 [2, 1, 13, 10, 6, 8, 4, 14]
18 [1, 2, 15, 12, 3, 8, 4, 16]
19 [1, 2, 12, 9, 3, 8, 5, 14]
20 [1, 2, -1, 7, 3, 8, 4, 13]
21 [0, 2, -1, 7, 4, 9, 5, 11]
22 [4, 1, -1, 8, 6, 9, 3, 11]
23 [2, 1, -1, 9, 8, 7, 3, 11]
24 [1, 3, -1, 9, 8, 4, 2, 11]
25 [3, 1, -1, 9, 8, 4, 2, 11]
26 [2, 1, -1, 9, 8, 4, 3, 11]
27 [3, 1, -1, 9, 6, 4, 2, 11]
28 [3, 1, -1, 8, 9, 5, 2, 12]
29 [2, 1, -1, 8, 12, 5, 4, 10]
30 [3, 1, -1, 8, 12, 4, 6, 9]
31 [3, 1, 14, 7, 12, 4, 5, 9]
32 [4, 1, 11, 9, 5, 2, 3, 12]
33 [4

[0, 1, 2, 3, 4, 5, 6, 7, 8]